**Imports**

In [1]:
# imports
from cshd import cshd_img_cube, get_timeseries_cshd_dataset, params_phenometrics, cube_query, cshd_array, calc_phenometrics
import os
import json
import numpy as np

# Import torch
import torch
import torch.nn as nn
import torch.optim as optim

**Sample data**

In [2]:
path_dir = os.path.dirname("")
with open(os.path.join(path_dir, 'timeseries_pheno_metrics.json')) as f:
    ts_json = json.load(f)

X_train = ts_json["timeseries_pheno_metrics"]
y_train = ts_json["label_id"]

**Convert data to PyTorch tensors**

In [3]:
X_train_tensor = torch.tensor(np.expand_dims(X_train, axis=1), dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)

print(X_train_tensor.shape)

torch.Size([862, 1, 41])


**LSTM classifier model**

In [4]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

**Define model parameters**

In [5]:
input_size = X_train_tensor.shape[2]
hidden_size = 64
num_layers = 2
output_size = len(set(y_train_tensor))

**Instantiate**

In [6]:
model = LSTMClassifier(input_size, hidden_size, num_layers, output_size)

**Loss function and optimizer**

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

**Train the model**

In [8]:
num_epochs = 1000
for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

Epoch [1/1000], Loss: 6.767248630523682
Epoch [2/1000], Loss: 6.75526762008667
Epoch [3/1000], Loss: 6.744091510772705
Epoch [4/1000], Loss: 6.732574939727783
Epoch [5/1000], Loss: 6.720937252044678
Epoch [6/1000], Loss: 6.709129333496094
Epoch [7/1000], Loss: 6.696915626525879
Epoch [8/1000], Loss: 6.68433141708374
Epoch [9/1000], Loss: 6.670986652374268
Epoch [10/1000], Loss: 6.657427787780762
Epoch [11/1000], Loss: 6.643556594848633
Epoch [12/1000], Loss: 6.629136085510254
Epoch [13/1000], Loss: 6.6139655113220215
Epoch [14/1000], Loss: 6.597780227661133
Epoch [15/1000], Loss: 6.580639839172363
Epoch [16/1000], Loss: 6.562892913818359
Epoch [17/1000], Loss: 6.544076919555664
Epoch [18/1000], Loss: 6.523852825164795
Epoch [19/1000], Loss: 6.502443313598633
Epoch [20/1000], Loss: 6.479214668273926
Epoch [21/1000], Loss: 6.455333709716797
Epoch [22/1000], Loss: 6.430228233337402
Epoch [23/1000], Loss: 6.40261173248291
Epoch [24/1000], Loss: 6.373445987701416
Epoch [25/1000], Loss: 6.34

**Prediction**

In [9]:
def return_class_by_id(id):
    if 370: return "Soja"
    if 372: "Arroz"
    if 359: "Vegetação Florestal"
    if 365: "Corpos d'agua"
    if 367: return "Superfícies Artificiais"
    if 363: return "Pastagem"
    if 361: return "Formação Campestre"

**Build S2 Data Cubes**

In [11]:
S2_NDVI_cube = cshd_img_cube(
    data_dir=os.path.join(path_dir,'021037/')
)

S2_NDVI_cube

<xarray.Dataset> Size: 10GB
Dimensions:      (band: 1, x: 10560, y: 10560, time: 23)
Coordinates:
  * band         (band) int64 8B 1
  * x            (x) float64 84kB 5.264e+06 5.264e+06 ... 5.37e+06 5.37e+06
  * y            (y) float64 84kB 7.941e+06 7.941e+06 ... 7.835e+06 7.835e+06
    spatial_ref  int64 8B 0
  * time         (time) datetime64[ns] 184B 2023-01-01 2023-01-17 ... 2023-12-19
Data variables:
    band_data    (time, band, y, x) float32 10GB -6.864e+03 ... -230.0

In [13]:
S2_SCL_cube = cshd_img_cube(
    data_dir=os.path.join(path_dir,'SCL_021037/')
)

S2_SCL_cube

<xarray.Dataset> Size: 10GB
Dimensions:      (band: 1, x: 10560, y: 10560, time: 23)
Coordinates:
  * band         (band) int64 8B 1
  * x            (x) float64 84kB 5.264e+06 5.264e+06 ... 5.37e+06 5.37e+06
  * y            (y) float64 84kB 7.941e+06 7.941e+06 ... 7.835e+06 7.835e+06
    spatial_ref  int64 8B 0
  * time         (time) datetime64[ns] 184B 2023-01-01 2023-01-17 ... 2023-12-19
Data variables:
    band_data    (time, band, y, x) float32 10GB 6.0 6.0 6.0 6.0 ... 6.0 6.0 6.0

**Config Phenometrics**

In [14]:
config = params_phenometrics(
    peak_metric='pos', 
    base_metric='vos', 
    method='seasonal_amplitude', 
    factor=0.2, 
    thresh_sides='two_sided', 
    abs_value=0.1
)

**S2 Cloud Mask Config**

In [15]:
cloud_dict = {
    'S2-16D-2':{
        'cloud_band': 'SCL',
        'non_cloud_values': [4,5,6],
        'cloud_values': [0,1,2,3,7,8,9,10,11]
    }
}

cloud = cloud_dict['S2-16D-2']

In [16]:
def create_filter_array(array, filter_true, filter_false):
    filter_arr = []
    for element in array:
        if element in filter_true:
            filter_arr.append(0)
        if element in filter_false:
            filter_arr.append(1)
    return filter_arr

**Interact S2 Data Cube**

In [ ]:
model_predictions = []

for x in S2_NDVI_cube['x'][:1100]:              
    for y in S2_NDVI_cube['y'][:1100]:

        # Get raw time series from cube
        data = get_timeseries_cshd_dataset(
            cube=S2_NDVI_cube, 
            geom=[dict(coordinates = [x.values[()], y.values[()]])]
        )
        raw_timeseries = data['values']

        # Get cloud time series from cloud cube
        cloud_data = get_timeseries_cshd_dataset(
            cube=S2_SCL_cube, 
            geom=[dict(coordinates = [x.values[()], y.values[()]])]
        )
        cloud_array = create_filter_array(cloud_data['values'], cloud['cloud_values'], cloud['non_cloud_values'])

        # Apply cloud mask on raw time series
        for i in range(len(data['values'])):
            if cloud_array[i] == 0:
                raw_timeseries[i] = -9999

        # Create a xarray for time series
        ndvi_array = cshd_array(
            timeserie=raw_timeseries,
            start_date='2023-01-01',
            freq='16D'
        )

        # Calculate phenological metrics from xarray time series
        ds_phenos = calc_phenometrics(
            da=ndvi_array,
            engine='phenolopy',
            config=config,
            start_date='2023-01-01'
        )

        # Build LSTM input array from time series and phenological metrics
        tl = list(raw_timeseries) + list(ds_phenos)

        # Create a torch tensor
        X_test_tensor = torch.tensor(np.expand_dims([tl], axis=1), dtype=torch.float32)

        # Use train model to predict a class based on argmax
        with torch.no_grad():
            predictions = model(X_test_tensor)
            predicted_labels = torch.argmax(predictions, dim=1)

            # Prediction
            model_predictions.append(dict(x=x.values[()],y=y.values[()], time= '2023', band_data=int(predicted_labels[0])))

with open(os.path.join(path_dir, "model_predictions_1-10.json"), 'w') as fp:
    json.dump(dict(model_predictions = model_predictions), fp)
        

In [ ]:
model_predictions = []

for x in S2_NDVI_cube['x'][1101:2200]:            
    for y in S2_NDVI_cube['y'][1101:2200]:

        # Get raw time series from cube
        data = get_timeseries_cshd_dataset(
            cube=S2_NDVI_cube, 
            geom=[dict(coordinates = [x.values[()], y.values[()]])]
        )
        raw_timeseries = data['values']

        # Get cloud time series from cloud cube
        cloud_data = get_timeseries_cshd_dataset(
            cube=S2_SCL_cube, 
            geom=[dict(coordinates = [x.values[()], y.values[()]])]
        )
        cloud_array = create_filter_array(cloud_data['values'], cloud['cloud_values'], cloud['non_cloud_values'])

        # Apply cloud mask on raw time series
        for i in range(len(data['values'])):
            if cloud_array[i] == 0:
                raw_timeseries[i] = -9999

        # Create a xarray for time series
        ndvi_array = cshd_array(
            timeserie=raw_timeseries,
            start_date='2023-01-01',
            freq='16D'
        )

        # Calculate phenological metrics from xarray time series
        ds_phenos = calc_phenometrics(
            da=ndvi_array,
            engine='phenolopy',
            config=config,
            start_date='2023-01-01'
        )

        # Build LSTM input array from time series and phenological metrics
        tl = list(raw_timeseries) + list(ds_phenos)

        # Create a torch tensor
        X_test_tensor = torch.tensor(np.expand_dims([tl], axis=1), dtype=torch.float32)

        # Use train model to predict a class based on argmax
        with torch.no_grad():
            predictions = model(X_test_tensor)
            predicted_labels = torch.argmax(predictions, dim=1)

            # Prediction
            model_predictions.append(dict(x=x.values[()],y=y.values[()], time= '2023', band_data=int(predicted_labels[0])))

with open(os.path.join(path_dir, "model_predictions_2-10.json"), 'w') as fp:
    json.dump(dict(model_predictions = model_predictions), fp)